<a href="https://colab.research.google.com/github/manavsrivastav14/NLP-miniproject/blob/main/Copy_of_NLP_MINI_PROJECT_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.0 MB/s 


In [ ]:
from fastai.text import *
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import sentencepiece as spm
import re
import pdb

In [ ]:
import fastai, torch
fastai.__version__ , torch.__version__

('1.0.61', '1.10.0+cu111')

In [ ]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [ ]:
random_seed(42, True)

In [ ]:
path = Path('./')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! gdown --id 1oRWNkYp3lU8Hq8kdmkddXEprObr177CD


/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1oRWNkYp3lU8Hq8kdmkddXEprObr177CD
To: /content/pa-train.csv
100% 10.2M/10.2M [00:00<00:00, 138MB/s]


In [ ]:
df_train = pd.read_csv(path/'/content/pa-train.csv', header=None)
df_train.head()

0                                                  1
0         sports  ⁄   ਕ੍ਰਿਕੇਟ\n  ⁄  ਜਨਰਲ\nInd vs Ban : ਚਾਰ ਸਾਲਾਂ...
1       business  ਸੈਂਸੈਕਸ ਦੀਆਂ ਟਾਪ 10 ਕੰਪਨੀਆਂ 'ਚੋਂ ਸੱਤ ਦਾ ਬਾਜ਼ਾਰ ...
2  entertainment  ਵਿੱਦਿਆ ਬਾਲਨ ਨੇ ਮਨਾਇਆ ਵਰਲਡ ਮੈਥਸ ਡੇ\nPublish Dat...
3         sports  ਆਈਐੱਸਐੱਲ-6 ਦਾ ਆਗਾਜ਼, ਬਲਾਸਟਰਜ਼ ਨੇ ਜਿੱਤਿਆ ਮੈਚ\nPub...
4  entertainment  ਬਚਪਨ ਦੇ ਦੋਸਤ ਲਈ ਰਣਵੀਰ ਸਿੰਘ ਨੇ ਬਦਲਿਆ ਆਪਣਾ ਸ਼ਡਿਊਲ...

In [ ]:
! gdown --id 1PvbCH67SmgdWLDp0m1kEz2C8PWLNwf_r

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1PvbCH67SmgdWLDp0m1kEz2C8PWLNwf_r
To: /content/pa-valid.csv
100% 1.27M/1.27M [00:00<00:00, 109MB/s]


In [ ]:
df_valid = pd.read_csv(path/'/content/pa-valid.csv', header=None)
df_valid.head()

0                                                  1
0  entertainment  ⁄   ਮਨੋਰੰਜਨ\n  ⁄   ਬਾਲੀਵੁੱਡ\nLady Gaga Viral V...
1  entertainment  ⁄   ਮਨੋਰੰਜਨ\n  ⁄   ਬਾਲੀਵੁੱਡ\nਨਸੀਰੂਦੀਨ ਸ਼ਾਹ ਦੇ ਬ...
2         sports  ਮਾਸਟਰ ਹੈੱਡ\nPublish Date:Mon, 20 Aug 2018 10:2...
3       business  ਟੀਵੀਐੱਸ ਮੋਟਰ ਦਾ ਮੁਨਾਫ਼ਾ ਵਧ ਕੇ ਹੋਇਆ 154.3 ਕਰੋੜ ਰ...
4       politics  ⁄   ਚੋਣਾਂ\n  ⁄  ਲੋਕ ਸਭਾ\nਦਰਿਆਈ ਪਾਣੀ ਹੋਇਆ ਜ਼ਹਿਰੀ...

In [ ]:
! gdown --id 186o_Jh8ufmsd_uSiSkzTZMGNPWURcHQh

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=186o_Jh8ufmsd_uSiSkzTZMGNPWURcHQh
To: /content/pa-test.csv
100% 1.32M/1.32M [00:00<00:00, 42.6MB/s]


In [ ]:
df_test = pd.read_csv('pa-test.csv', header=None)
df_test.head()

0                                                  1
0       politics  ਪ੍ਰਸ਼ਾਸਨ ਨੇ ਡਾ. ਧਰਮਵੀਰ ਗਾਂਧੀ ਦੇ ਨਾਂ 'ਤੇ ਲਾਈ ਚੇਪ...
1         sports  ਹਰ ਸਮੇਂ ਨਾਲ ਨਹੀਂ ਰਹੇਗਾ ਖਿਡਾਰੀਆਂ ਦਾ ਪਰਿਵਾਰ\nPub...
2       business  4ਜੀ ਨੂੰ ਪਛਾੜ ਐੱਲਈਡੀ ਬਲਬ ਦੇਵੇਗਾ ਹਾਈ ਸਪੀਡ ਇੰਟਰਨੈ...
3  entertainment  ਪੀਐਮ ਮੋਦੀ ਦੇ ਬਚਪਨ ’ਤੇ ਬਣੀ ਫਿ਼ਲਮ ‘ਚਲੋ ਜੀਤੇ ਹੈਂ’...
4         sports  ਬਹੁਤ ਚੰਗਾ ਮਹਿਸੂਸ ਹੋ ਰਿਹੈ : ਕੋਹਲੀ\nPublish Date...

In [ ]:
df_train.shape, df_valid.shape, df_test.shape

((2496, 2), (312, 2), (312, 2))

In [ ]:
df_train[df_train[0].isnull()].shape, df_valid[df_valid[0].isnull()].shape, df_test[df_test[0].isnull()].shape

((0, 2), (0, 2), (0, 2))

In [ ]:
label_cols = [0]

In [ ]:
class PanjabiTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str('panjabi_lm.model'))
        
    def tokenizer(self, t:str) -> List[str]:
        return self.sp.EncodeAsPieces(t)

In [ ]:
! gdown --id 16y0GO7fglAMRl3mB-tUa2kY1kRjzyM6i

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=16y0GO7fglAMRl3mB-tUa2kY1kRjzyM6i
To: /content/panjabi_lm.model
100% 4.22M/4.22M [00:00<00:00, 74.1MB/s]


In [ ]:
sp = spm.SentencePieceProcessor()
sp.Load('/content/panjabi_lm.model')
itos = [sp.IdToPiece(int(i)) for i in range(30000)]

In [ ]:
# 30,000 is the vocab size that we chose in sentencepiece
panjabi_vocab = Vocab(itos)

In [ ]:
panjabi_tok = PanjabiTokenizer('pb')

In [ ]:
tokenizer = Tokenizer(tok_func=PanjabiTokenizer, lang='pb')

In [ ]:
tokenizer.special_cases

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep']

In [ ]:
data_lm = TextLMDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=panjabi_vocab, label_cols=label_cols)

/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(a, dtype=dtype, **kwargs)


In [ ]:
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.3, pretrained=False)

In [ ]:
# Loading the pretrained language model on panjabi wikipedia
learn.load('/content/drive/MyDrive/NLP/model', with_opt=True)

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (2496 items)
x: LMTextList
▁x x bo s ▁ ⁄ ▁ਕ੍ਰਿਕੇਟ ▁ ⁄ ▁ਜਨਰਲ ▁in d ▁vs <unk> ▁b an ▁: ▁ਚਾਰ ▁ਸਾਲਾਂ ▁' ਚ ▁ਖੇਡੇ ▁ਗਏ ▁ਸਿਰਫ਼ ▁11 <unk> - n ight ▁ਟੈਸਟ ▁ਮੈਚ , ▁ਭਾਰਤ ▁ਬਣੇਗਾ ▁ਅਜਿਹਾ ▁ਕਰਨ ▁ਵਾਲਾ ▁7 ਵਾਂ ▁ਦੇਸ਼ <unk> ▁p ub lish <unk> ▁d ate : t hu , ▁31 <unk> ▁o ct ▁ 2019 ▁0 9 : 13 <unk> <unk> ▁( i <unk> st ) ▁ਭਾਰਤੀ ▁ਕ੍ਰਿਕਟ ▁ਕੰਟਰੋਲ ▁ਬੋਰਡ ▁ਦੇ ▁ਪ੍ਰਧਾਨ ▁ਸੌਰ ਵ ▁ਗਾਂਗੁਲੀ ▁ਦੇ ▁ ਡੇਅ - ਨਾਈਟ ▁ਟੈਸਟ ▁ਦੇ ▁ਭਾਰਤ ▁' ਚ ▁' ਚ ▁ਕਰਵਾਉਣ ▁ਦੇ ▁ਫ਼ੈਸਲੇ ▁ਨੇ ▁ਨਵਾਂ ▁ਇਤਿਹਾਸ ▁ਰਚਿਆ ▁ਹੈ । ▁ਨਵੀਂ ▁ਦਿੱਲੀ , ▁ਜੇ ਐੱਨ ਐੱਨ ▁: ▁ਭਾਰਤੀ ▁ਕ੍ਰਿਕਟ ▁ਕੰਟਰੋਲ ▁ਬੋਰਡ ▁ਦੇ ▁ਪ੍ਰਧਾਨ ▁ਸੌਰ ਵ ▁ਗਾਂਗੁਲੀ ▁ਦੇ ▁ ਡੇਅ - ਨਾਈਟ ▁ਟੈਸਟ ▁ਦੇ ▁ਭਾਰਤ ▁' ਚ ▁' ਚ ▁ਕਰਵਾਉਣ ▁ਦੇ ▁ਫ਼ੈਸਲੇ ▁ਨੇ ▁ਨਵਾਂ ▁ਇਤਿਹਾਸ ▁ਰਚਿਆ ▁ਹੈ । ▁ਇਸ ▁ਫ਼ੈਸਲੇ ▁ਤੋਂ ▁ਬਾਅਦ ▁ਹੁਣ ▁ਭਾਰਤ ▁ਵੀ ▁ਉਨ੍ਹਾਂ ▁ਟੀਮਾਂ ▁ਦੀ ▁ਸੂਚੀ ▁' ਚ ▁ਸ਼ਾਮਲ ▁ਹੋ ▁ਜਾਵੇਗਾ , ▁ਜਿਨ੍ਹਾਂ ▁ਨੇ ▁ਟੈਸਟ ▁ਕ੍ਰਿਕਟ ▁ਦੇ ▁ਇਸ ▁ਨਵੇਂ ▁ਰੋਮ ਾਂ ਚ ▁' ਚ ▁ਸ਼ਿਰਕ ਤ ▁ਕੀਤੀ ▁ਹੈ । ▁ਸੰਨ ▁2015 ▁' ਚ ▁ਨਵੰਬਰ ▁' ਚ ▁ਪਹਿਲੀ ▁ਵਾਰ ▁ਇੰਟਰਨੈਸ਼ਨਲ ▁ ਡੇਅ - ਨਾਈਟ ▁ਟੈਸਟ ▁ਮੈਚ ▁ਖੇਡਿਆ ▁ਗਿਆ ▁ਸੀ ▁ਪਰ ▁ਉਦੋਂ ▁ਤੋਂ ▁ਹੁਣ ▁ਤਕ ▁ਸਿਰਫ਼ ▁11 ▁ਮੈਚ ▁ਹੀ ▁ਖੇਡੇ ▁ਜਾ ▁ਸਕੇ ▁ਹ

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
learn.predict('ਜੋ ਉਹਨਾਂ ਦੇ ਰੱਬਾਂ ਨੂੰ ਪ੍ਰਸਤੁਤ ਕਰਦੇ',n_words=10)

'ਜੋ ਉਹਨਾਂ ਦੇ ਰੱਬਾਂ ਨੂੰ ਪ੍ਰਸਤੁਤ ਕਰਦੇ ▁ਹਨ ▁ਅਤੇ ▁ਕੁਦਰਤ ▁ਦਾ ▁ਅੰਦਰ ▁ਜਾਣ ੇਗਾ ▁ਇਸ ▁ਮੇਲੇ ▁ਨੂੰ'

In [ ]:
learn.save_encoder('fine_tuned_enc')

In [ ]:
data_clas = TextClasDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=panjabi_vocab, label_cols=label_cols, bs=64)

/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(a, dtype=dtype, **kwargs)


In [ ]:
data_clas.show_batch()

In [ ]:
learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.5)

In [ ]:
learn.load_encoder('fine_tuned_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (2496 items)
x: TextList
▁x x bo s ▁ ⁄ ▁ਕ੍ਰਿਕੇਟ ▁ ⁄ ▁ਜਨਰਲ ▁in d ▁vs <unk> ▁b an ▁: ▁ਚਾਰ ▁ਸਾਲਾਂ ▁' ਚ ▁ਖੇਡੇ ▁ਗਏ ▁ਸਿਰਫ਼ ▁11 <unk> - n ight ▁ਟੈਸਟ ▁ਮੈਚ , ▁ਭਾਰਤ ▁ਬਣੇਗਾ ▁ਅਜਿਹਾ ▁ਕਰਨ ▁ਵਾਲਾ ▁7 ਵਾਂ ▁ਦੇਸ਼ <unk> ▁p ub lish <unk> ▁d ate : t hu , ▁31 <unk> ▁o ct ▁ 2019 ▁0 9 : 13 <unk> <unk> ▁( i <unk> st ) ▁ਭਾਰਤੀ ▁ਕ੍ਰਿਕਟ ▁ਕੰਟਰੋਲ ▁ਬੋਰਡ ▁ਦੇ ▁ਪ੍ਰਧਾਨ ▁ਸੌਰ ਵ ▁ਗਾਂਗੁਲੀ ▁ਦੇ ▁ ਡੇਅ - ਨਾਈਟ ▁ਟੈਸਟ ▁ਦੇ ▁ਭਾਰਤ ▁' ਚ ▁' ਚ ▁ਕਰਵਾਉਣ ▁ਦੇ ▁ਫ਼ੈਸਲੇ ▁ਨੇ ▁ਨਵਾਂ ▁ਇਤਿਹਾਸ ▁ਰਚਿਆ ▁ਹੈ । ▁ਨਵੀਂ ▁ਦਿੱਲੀ , ▁ਜੇ ਐੱਨ ਐੱਨ ▁: ▁ਭਾਰਤੀ ▁ਕ੍ਰਿਕਟ ▁ਕੰਟਰੋਲ ▁ਬੋਰਡ ▁ਦੇ ▁ਪ੍ਰਧਾਨ ▁ਸੌਰ ਵ ▁ਗਾਂਗੁਲੀ ▁ਦੇ ▁ ਡੇਅ - ਨਾਈਟ ▁ਟੈਸਟ ▁ਦੇ ▁ਭਾਰਤ ▁' ਚ ▁' ਚ ▁ਕਰਵਾਉਣ ▁ਦੇ ▁ਫ਼ੈਸਲੇ ▁ਨੇ ▁ਨਵਾਂ ▁ਇਤਿਹਾਸ ▁ਰਚਿਆ ▁ਹੈ । ▁ਇਸ ▁ਫ਼ੈਸਲੇ ▁ਤੋਂ ▁ਬਾਅਦ ▁ਹੁਣ ▁ਭਾਰਤ ▁ਵੀ ▁ਉਨ੍ਹਾਂ ▁ਟੀਮਾਂ ▁ਦੀ ▁ਸੂਚੀ ▁' ਚ ▁ਸ਼ਾਮਲ ▁ਹੋ ▁ਜਾਵੇਗਾ , ▁ਜਿਨ੍ਹਾਂ ▁ਨੇ ▁ਟੈਸਟ ▁ਕ੍ਰਿਕਟ ▁ਦੇ ▁ਇਸ ▁ਨਵੇਂ ▁ਰੋਮ ਾਂ ਚ ▁' ਚ ▁ਸ਼ਿਰਕ ਤ ▁ਕੀਤੀ ▁ਹੈ । ▁ਸੰਨ ▁2015 ▁' ਚ ▁ਨਵੰਬਰ ▁' ਚ ▁ਪਹਿਲੀ ▁ਵਾਰ ▁ਇੰਟਰਨੈਸ਼ਨਲ ▁ ਡੇਅ - ਨਾਈਟ ▁ਟੈਸਟ ▁ਮੈਚ ▁ਖੇਡਿਆ ▁ਗਿਆ ▁ਸੀ ▁ਪਰ ▁ਉਦੋਂ ▁ਤੋਂ ▁ਹੁਣ ▁ਤਕ ▁ਸਿਰਫ਼ ▁11 ▁ਮੈਚ ▁ਹੀ ▁ਖੇਡੇ ▁ਜਾ ▁ਸਕੇ ▁ਹਨ । ▁

In [ ]:
learn.loss_func.func

CrossEntropyLoss()

In [ ]:
mcc = MatthewsCorreff()

In [ ]:
learn.metrics = [accuracy, mcc]

In [ ]:
learn.fit_one_cycle(2, 1e-2, callbacks=[callbacks.SaveModelCallback(learn, every='improvement', monitor='accuracy', name='final')])

Better model found at epoch 0 with accuracy value: 0.9679487347602844.


In [ ]:
from keras.models import load_model
learn.save('My_Model.h5')
del learn

In [ ]:
from sklearn.metrics import accuracy_score, matthews_corrcoef
df_dict = {'query': list(df_test[1]), 'actual_label': list(df_test[0]), 'predicted_label': ['']*df_test.shape[0]}
all_nodes = list(set(df_train[0]))
for node in all_nodes:
    df_dict[node] = ['']*df_test.shape[0]
    
i2c = {}
for key, value in learn.data.c2i.items():
    i2c[value] = key
    
df_result = pd.DataFrame(df_dict)
preds = learn.get_preds(ds_type=DatasetType.Test, ordered=True)
for index, row in df_result.iterrows():
    for node in all_nodes:
        row[node] = preds[0][index][learn.data.c2i[node]].item()
    row['predicted_label'] = i2c[np.argmax(preds[0][index]).data.item()]
df_result.head()

NameError: ignored

In [ ]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])

0.9583333333333334

In [ ]:
matthews_corrcoef(df_result['actual_label'], df_result['predicted_label'])

0.9448994337320976

In [ ]:
df_result.to_csv('indicnlp_news_articles_pa_result.csv', index=False)